# Task 1

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd 
import numpy as np 
from pyspark.sql.functions import split, col, regexp_replace, collect_list, explode, concat, collect_set, array_union, flatten
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType, BooleanType

import time
import random
import subprocess

### Configure Spark

In [2]:
partition = 200

In [3]:
spark = SparkSession.builder \
    .appName("Projet-Task-1") \
    .master("local[*]") \
    .config("spark.driver.memory", "12G") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.executor.memory", "6G") \
    .config("spark.executor.memoryOverhead", "2G") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:MaxGCPauseMillis=500 -XX:InitiatingHeapOccupancyPercent=35") \
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC -XX:MaxGCPauseMillis=500 -XX:InitiatingHeapOccupancyPercent=35") \
    .getOrCreate()
# spark.sparkContext.setLogLevel("DEBUG")
spark.conf.set("spark.sql.shuffle.partitions", partition)
spark

24/06/25 23:55:13 WARN Utils: Your hostname, abha-ThinkPad-P14s-Gen-4 resolves to a loopback address: 127.0.1.1; using 192.168.178.94 instead (on interface wlp2s0)
24/06/25 23:55:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/25 23:55:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# spark.stop()

### Read Data and Clean Logs

In [5]:
df = spark.read.text("data.txt").toDF("Log")

In [6]:
df = df.withColumn("Log", regexp_replace(col("Log"), "[<>]", ""))
df = df.withColumn("Log", regexp_replace(col("Log"), ",", ""))
df = df.withColumn("Log", split(col("Log"), " "))

columns = ["First Server", "Second Server", "Communication Type", "Process ID"]

for i in range(len(columns)):
    # print(columns[i])
    df = df.withColumn(columns[i], col("Log")[i])

df.printSchema()
# df.show()

root
 |-- Log: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- First Server: string (nullable = true)
 |-- Second Server: string (nullable = true)
 |-- Communication Type: string (nullable = true)
 |-- Process ID: string (nullable = true)



In [7]:
# Remove Process ID from request 
log = udf(lambda x: x[:-1], ArrayType(StringType())) 
df = df.withColumn('Log', log('Log')) 

### Group Log Events by Process ID

In [8]:
grouped_df = df.groupBy("Process ID").agg(collect_list("Log").alias("Log"), 
                                          collect_list("First Server").alias("First Server"),
                                          collect_list("Second Server").alias("Second Server"),
                                          collect_list("Communication Type").alias("FCommunication Type"))

grouped_df.printSchema()
# grouped_df.show()

root
 |-- Process ID: string (nullable = true)
 |-- Log: array (nullable = false)
 |    |-- element: array (containsNull = false)
 |    |    |-- element: string (containsNull = true)
 |-- First Server: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- Second Server: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- FCommunication Type: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [9]:
distinct_servers_df = df.groupBy("Process ID").agg(collect_set("First Server").alias("First Server"),
                                                   collect_set("Second Server").alias("Second Server"))

distinct_servers_df = distinct_servers_df.withColumn("Servers", array_union("First Server", "Second Server"))

distinct_servers_df.printSchema()
# distinct_servers_df.show()

root
 |-- Process ID: string (nullable = true)
 |-- First Server: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- Second Server: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- Servers: array (nullable = false)
 |    |-- element: string (containsNull = false)



### Build Characteristic Matrix 

In [10]:
characteristics = CountVectorizer(inputCol="Servers", outputCol="Characteristic Matrix")

model = characteristics.fit(distinct_servers_df)
char_matrix = model.transform(distinct_servers_df).select("Process ID", "Characteristic Matrix")

char_matrix.printSchema()
# char_matrix.show()

servers = model.vocabulary
# print("Rows of Characteristic Matrix: ", servers)

root
 |-- Process ID: string (nullable = true)
 |-- Characteristic Matrix: vector (nullable = true)



### Generate MinHash

In [11]:
minhash = MinHashLSH(inputCol="Characteristic Matrix", outputCol="Signatures", numHashTables=5)

# MinHash produces the signatures for the Characteritic matrix 
# numvHashTables is the number of the hash functioms that we want to use and the lenght of the signature 
model = minhash.fit(char_matrix)
signatures = model.transform(char_matrix)

# signatures.show()

### Find Similar Pairs

In [12]:
# approxSimilarityJoin uses LSH automatically to find rows that it is most likely 
# to have same "Signatures"
# threshold: pairs with Jaccard Distance lower than threshlod
similar_pairs = model.approxSimilarityJoin(signatures, signatures, threshold=0.01, distCol="Jaccard Distance")
# similar_pairs.show()

In [13]:
similar_pairs = similar_pairs.select("datasetA.Process ID", "datasetB.Process ID", 
                     "Jaccard Distance") \
                    # .filter((col("datasetA.Process ID") != col("datasetB.Process ID")))

In [14]:
new_cols = ["Process ID A", "Process ID B", "Jaccard Distance"]
similar_pairs = similar_pairs.toDF(*new_cols)

In [15]:
# similar_pairs.show()

In [16]:
pairs = similar_pairs.join(grouped_df, similar_pairs["Process ID A"] == col("Process ID")) \
                     .select(col("Process ID A"), col("Process ID B"), col("Log").alias("Log A")) \
                     .join(grouped_df, similar_pairs["Process ID B"] == col("Process ID")) \
                     .select(col("Process ID A"), col("Process ID B"), col("Log A"), col("Log").alias("Log B"))

In [17]:
# pairs.show()

### Check using Original Log

In [18]:
def original_check(x,y):
    return x==y

original_checking = udf(original_check, BooleanType())

In [19]:
same_pairs = pairs.filter(original_checking(col("Log A"), col("Log B")))
same_pairs = same_pairs.groupBy("Log A").agg(collect_set("Process ID A").alias("Process Set"))

In [20]:
same_pairs.printSchema()

root
 |-- Log A: array (nullable = false)
 |    |-- element: array (containsNull = false)
 |    |    |-- element: string (containsNull = true)
 |-- Process Set: array (nullable = false)
 |    |-- element: string (containsNull = false)



### Output

part1Observations.txt

In [21]:
same_pairs_explode = same_pairs.select(same_pairs["Log A"], same_pairs["Process Set"], explode(same_pairs["Process Set"]).alias("Process ID"))

In [22]:
def format_group(process_set):
    process_set_string = ', '.join(str(x) for x in process_set)
    return f"Group: {{{process_set_string}}}"

def format_log(log, process_id):
    log_formatted = ""
    for l in log:
        log_concat = ', '.join(str(x) for x in l)
        log_formatted += f"<{log_concat}, {process_id}>\n"
    return log_formatted

def format_group_logs(group, logs):
    formatted = f"{group}\n\n" + "\n".join(logs) 
    return formatted

In [23]:
# UDFs for Formatting Output - part1Observations.txt
format_group_udf = udf(format_group, StringType())
format_udf = udf(format_log, StringType())
final_format_udf = udf(format_group_logs, StringType())

In [24]:
formatted_group = same_pairs_explode.withColumn("Group", format_group_udf(col("Process Set")))
formatted_df = formatted_group.withColumn("Formatted Log", format_udf(col("Log A"), col("Process ID")))
grouped_logs = formatted_df.groupBy("Group").agg(collect_list("Formatted Log").alias("Group Log"))
final_formatted = grouped_logs.withColumn("Formatted", final_format_udf(col("Group"), col("Group Log"))).select("Formatted")

In [25]:
final_formatted.coalesce(partition).write.mode('overwrite').text('part1Observations')

In [26]:
subprocess.run("mkdir -p output && cat part1Observations/part-* > output/part1Observations.txt", shell=True)
subprocess.run("find part1Observations/ -name 'part-*' -delete", shell=True)
subprocess.run("rm -f part1Observations/.*", shell=True)
subprocess.run("rm -f part1Observations/_SUCCESS", shell=True) 
subprocess.run("rmdir part1Observations", shell=True)

rm: cannot remove 'part1Observations/.': Is a directory
rm: cannot remove 'part1Observations/..': Is a directory


CompletedProcess(args='rmdir part1Observations', returncode=0)

part1Output.txt

In [27]:
from pyspark.sql.types import IntegerType

df = df.withColumn('pid_integer', df['Process ID'].cast(IntegerType()))
max_process_id = df.agg({"pid_integer": "max"}).collect()[0][0]
# print(max_process_id)

In [28]:
def format_log_output(log):
    epoch = int(time.time())
    rand = random.randint(1000, 9999)
    process_id = f"{epoch}{rand}"
    log_formatted = ""
    for l in log:
        log_concat = ', '.join(str(x) for x in l)
        log_formatted += f"<{log_concat}, {process_id}>\n"

    formatted = f"{process_id}:\n" + log_formatted
    return formatted

format_udf_output = udf(format_log_output, StringType())
formatted_df_output = same_pairs.withColumn("Formatted", format_udf_output(col("Log A"))).select("Formatted")

In [29]:
formatted_df_output.coalesce(partition).write.mode('overwrite').text('part1Output')

In [30]:
subprocess.run("cat part1Output/part-* > output/part1Output.txt", shell=True)
subprocess.run("find part1Output/ -name 'part-*' -delete", shell=True)
subprocess.run("rm -f part1Output/.*", shell=True)
subprocess.run("rm -f part1Output/_SUCCESS", shell=True) 
subprocess.run("rmdir part1Output", shell=True)

rm: cannot remove 'part1Output/.': Is a directory
rm: cannot remove 'part1Output/..': Is a directory


CompletedProcess(args='rmdir part1Output', returncode=0)

## Another aproach - Shingling

In [31]:
def k_shingling(text, k):
    shingles = set()
    for i in range(len(text) - k + 1):
        shingle = text[i:i + k]
        shingles.add(shingle)
    return list(shingles)

k_shingling_udf = udf(lambda text: k_shingling(text, 5), ArrayType(StringType()))

In [32]:
df = spark.read.text("data.txt").toDF("Log")
log = udf(lambda x: x[1:-4], StringType()) 
df = df.withColumn('Log_split', log('Log')) 
df.collect()
df_shingles = df.withColumn("Shingles", k_shingling_udf(df["Log_split"]))
# df_shingles.show()

In [33]:
df_shingles = df_shingles.withColumn("Log", regexp_replace(col("Log"), "[<>]", ""))
df_shingles = df_shingles.withColumn("Log", regexp_replace(col("Log"), ",", ""))
df_shingles = df_shingles.withColumn("Log", split(col("Log"), " "))

df_shingles.printSchema()
# df_shingles.show()

columns = ["First Server", "Second Server", "Communication Type", "Process ID"]

for i in range(len(columns)):
    print(columns[i])
    df_shingles = df_shingles.withColumn(columns[i], col("Log")[i])

df_shingles.printSchema()
# df_shingles.show()

root
 |-- Log: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- Log_split: string (nullable = true)
 |-- Shingles: array (nullable = true)
 |    |-- element: string (containsNull = true)

First Server
Second Server
Communication Type
Process ID
root
 |-- Log: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- Log_split: string (nullable = true)
 |-- Shingles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- First Server: string (nullable = true)
 |-- Second Server: string (nullable = true)
 |-- Communication Type: string (nullable = true)
 |-- Process ID: string (nullable = true)



In [34]:
log = udf(lambda x: x[:-1], ArrayType(StringType())) 
df_shingles = df_shingles.withColumn('Log', log('Log')) 

In [35]:
grouped_df = df_shingles.groupBy("Process ID").agg(collect_set("Shingles").alias("Shingles"),collect_list("Log").alias("Log"))
grouped_df = grouped_df.withColumn("Flat shingles", flatten(col("Shingles")))
grouped_df.printSchema()

root
 |-- Process ID: string (nullable = true)
 |-- Shingles: array (nullable = false)
 |    |-- element: array (containsNull = false)
 |    |    |-- element: string (containsNull = true)
 |-- Log: array (nullable = false)
 |    |-- element: array (containsNull = false)
 |    |    |-- element: string (containsNull = true)
 |-- Flat shingles: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [36]:
characteristics = CountVectorizer(inputCol="Flat shingles", outputCol="Characteristic Matrix")

model = characteristics.fit(grouped_df)
char_matrix = model.transform(grouped_df).select("Process ID", "Characteristic Matrix")

char_matrix.printSchema()
# char_matrix.show()

shingles = model.vocabulary
print("Rows of Characteristic Matrix: ", shingles)

root
 |-- Process ID: string (nullable = true)
 |-- Characteristic Matrix: vector (nullable = true)

Rows of Characteristic Matrix:  ['eques', 'Respo', ', Res', 'Reque', ' Resp', 'quest', 'spons', 'ponse', ', Req', ' Requ', 'espon', ', S-1', '3, Re', '.3, R', ' S-1.', 'null,', 'ull, ', '3, S-', '.3, S', 'onse,', 'uest,', '1.3, ', 'S-1.3', '-1.3,', '.2, R', '1.1, ', '2, Re', '1.2, ', 'S-1.1', '-1.1,', '1, Re', '.1, R', '-1.2,', 'S-1.2', '2, S-', '.2, S', ', S-2', '.1, S', '1, S-', ' null', 'l, Re', 'll, S', 'll, R', ', nul', 'l, S-', ', S-4', ', S-3', '0.3, ', '.4, R', '4, Re', '.4, S', '5.3, ', 'S-20.', '4, S-', '44.4,', 'S-40.', '4.4, ', 'S-35.', '2, nu', '-40.3', '9.3, ', '-44.4', 'S-32.', 'S-23.', 'S-44.', '40.3,', '.2, n', '5.1, ', '.3, n', '3, nu', '29.3,', '48.2,', '20.1,', '2.3, ', 'S-36.', 'S-29.', '6.4, ', '35.3,', '-29.3', '-48.2', 'S-48.', '0.1, ', '4.3, ', 'S-41.', '2.2, ', 'S-19.', '8.2, ', '9.4, ', '19.4,', '-35.3', '-20.1', '-19.4', '1, nu', ' S-20', '.1, n', '8.3, ', ' 

In [37]:
minhash = MinHashLSH(inputCol="Characteristic Matrix", outputCol="Signatures", numHashTables=5)

# MinHash produces the signatures for the Characteritic matrix 
# numvHashTables is the number of the hush functioms that we want to use and the lenght of the signature 
model = minhash.fit(char_matrix)
signatures = model.transform(char_matrix)

# signatures.show()

In [38]:
# approxSimilarityJoin uses autmatically LSH to find rows that it is most likely 
# to have same "Signatures"
# threshold: pairs with Jaccard Distance lower than threshlod
similar_pairs = model.approxSimilarityJoin(signatures, signatures, threshold=0.2, distCol="Jaccard Distance")
# similar_pairs.show()

In [39]:
similar_pairs = similar_pairs.select("datasetA.Process ID", "datasetB.Process ID", 
                     "Jaccard Distance")\
                        # .filter((col("datasetA.Process ID") != col("datasetB.Process ID")))

In [40]:
new_cols = ["Process ID A", "Process ID B", "Jaccard Distance"]
similar_pairs = similar_pairs.toDF(*new_cols)

In [41]:
pairs = similar_pairs.join(grouped_df, similar_pairs["Process ID A"] == col("Process ID")) \
                     .select(col("Process ID A"), col("Process ID B"), col("Log").alias("Log A")) \
                     .join(grouped_df, similar_pairs["Process ID B"] == col("Process ID")) \
                     .select(col("Process ID A"), col("Process ID B"), col("Log A"), col("Log").alias("Log B"))

In [42]:
def original_check(x,y):
    return x==y

orifinal_checking = udf(original_check, BooleanType())
same_pairs = pairs.filter(orifinal_checking(col("Log A"), col("Log B")))

In [43]:
same_pairs = same_pairs.groupBy("Log A").agg(collect_set("Process ID A"))

24/06/25 23:55:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
